In [1]:
# import packages and set options
from formatting_functions_open_source import *
from utility_functions import *
import numpy as np
import pandas as pd
import xlsxwriter
import pandas.io.formats.excel

# this allows custom formatting of header
pandas.io.formats.excel.ExcelFormatter.header_style = None


In [2]:
# load in data
twoD_df = pd.read_csv('Data/avg_medical_svc_cost.csv')

threeD_row_df_1 = pd.read_csv('Data/clients_by_gender.csv')

threeD_row_df_2 = pd.read_csv('Data/clients_by_gender_2.csv')

test_dtype_df = pd.read_csv('Data/test_set_datatype.csv')

test_dtype_multi_df =pd.read_csv('Data/test_set_datatype_multiindex.csv')


In [3]:
# set indices
twoD_df.set_index('Department', inplace=True)

threeD_row_df_1.set_index(['Department', 'Gender'], inplace=True)

threeD_row_df_2.set_index(['Department','Gender','Trans'], inplace=True)

test_dtype_df.set_index('Department', inplace=True)

test_dtype_multi_df.set_index(['Department', 'Gender'], inplace=True)

In [4]:
test_dtype_multi_df['date_todt'] = pd.to_datetime(test_dtype_multi_df['date_todt'])
test_dtype_multi_df['date_alt_todt'] = pd.to_datetime(test_dtype_multi_df['date_alt_todt'])
test_dtype_multi_df['datetime_todt'] = pd.to_datetime(test_dtype_multi_df['datetime_todt'])
test_dtype_multi_df['datetime_alt_todt'] = pd.to_datetime(test_dtype_multi_df['datetime_alt_todt'])

test_dtype_multi_df['date_todt'] = pd.to_datetime(test_dtype_multi_df['date_todt'])
test_dtype_multi_df['date_alt_todt'] = pd.to_datetime(test_dtype_multi_df['date_alt_todt'])
test_dtype_multi_df['datetime_todt'] = pd.to_datetime(test_dtype_multi_df['datetime_todt'])
test_dtype_multi_df['datetime_alt_todt'] = pd.to_datetime(test_dtype_multi_df['datetime_alt_todt'])

In [5]:
# create excel report

# nan_inf_to_errors will prevent nans from breaking report creation
## but any nans in the report are a bug that will need to be fixed

writer = pd.ExcelWriter('Reports/Example Clinical Report.xlsx', engine='xlsxwriter', options={'nan_inf_to_errors':True})

# exporting analyses to excel sheets
#twoD_df.to_excel(writer, sheet_name='Avg Svc Cost by Dept')

# creating workbook with sheets
ecr_wb = writer.book

twoD_sheet = ecr_wb.add_worksheet('Avg Svc Cost by Dept')

threeDrow_sheet1 = ecr_wb.add_worksheet('Clients by Dept & Gender')

threeDrow_sheet2 = ecr_wb.add_worksheet('Clients by Dept & Gender NEW')

test_dtype_sheet = ecr_wb.add_worksheet('Data Type Test')

test_dtype_multi_sheet = ecr_wb.add_worksheet('Data Type Multiindex Test')


# FORMATTING AVG SVC COST BY DEPT (2 dimensional data)

# header
last_col_highlight_header(twoD_df, ecr_wb, twoD_sheet, header_offset=2)

# index
format_index(twoD_df, ecr_wb, twoD_sheet, header_offset=2)

# data
insert_data(twoD_df, ecr_wb, twoD_sheet, header_offset=2, data_type='dollar_cents')

set_column_widths(twoD_df, ecr_wb, twoD_sheet)

# borders
table_bottom_border(twoD_df, ecr_wb, twoD_sheet, header_offset=2)

table_right_border(twoD_df, ecr_wb, twoD_sheet, header_offset=2)

# title
insert_title(twoD_df, ecr_wb, twoD_sheet, 'Avg Service Cost by Department')


# FORMATTING CLIENTS BY DEPT & GENDER (3 dimensional data with row mulitiindex)

# header
last_col_highlight_header(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2)

# index
merge_row_index_cells(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2)

format_row_multiindex(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2)

# data
insert_row_multiindex_data(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2, data_type='numeric')

set_column_widths(threeD_row_df_1, ecr_wb, threeDrow_sheet1)

# borders
table_bottom_border(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2)

table_right_border(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2)

# title
insert_title(threeD_row_df_1, ecr_wb, threeDrow_sheet1, 'Unique Clients by Department & Gender')


# FORMATTING CLIENTS BY DEPT & GENDER NEW (3 dimensional data with row mulitiindex)

# header
last_col_highlight_header(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2)

# index
merge_row_index_cells(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2)

format_row_multiindex(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2)

# data
insert_row_multiindex_data(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2, data_type='numeric')

set_column_widths(threeD_row_df_2, ecr_wb, threeDrow_sheet2)

# borders
table_bottom_border(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2)

table_right_border(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2)

# title
insert_title(threeD_row_df_2, ecr_wb, threeDrow_sheet2, 'Unique Clients by Department & Gender (New Gender Classification Method)')


# dtype test 
format_header(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, header_offset=2)
merge_row_index_cells(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, header_offset=2)
format_row_multiindex(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, header_offset=2)
insert_row_multiindex_data(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, header_offset=2)

# drum rollllll
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'numeric', 'numeric', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'decimal_1', 'decimal_1', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'decimal_2', 'decimal_2', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'dollar', 'dollar', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'dollar_cents', 'dollar_cents', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'percent', 'percent', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'percent_1', 'percent_1', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'percent_2', 'percent_2', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'date', 'date', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'date_alt', 'date_alt', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'datetime', 'datetime', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'datetime_alt', 'datetime_alt', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'date_todt', 'date', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'date_alt_todt', 'date_alt', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'datetime_todt', 'datetime', col_width_method='all')
set_col_data_type(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'datetime_alt_todt', 'datetime_alt', col_width_method='all')



table_bottom_border(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, header_offset=2)
table_right_border(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, header_offset=2)
insert_title(test_dtype_multi_df, ecr_wb, test_dtype_multi_sheet, 'Data Type Multiindex Test')


# dtype test 
format_header(test_dtype_df, ecr_wb, test_dtype_sheet, header_offset=2)
format_index(test_dtype_df, ecr_wb, test_dtype_sheet, header_offset=2)
insert_data(test_dtype_df, ecr_wb, test_dtype_sheet, header_offset=2)

# drum rollllll
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'numeric', 'numeric', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'decimal_1', 'decimal_1', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'decimal_2', 'decimal_2', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'dollar', 'dollar', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'dollar_cents', 'dollar_cents', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'percent', 'percent', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'percent_1', 'percent_1', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'percent_2', 'percent_2', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'date', 'date', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'date_alt', 'date_alt', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'datetime', 'datetime', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'datetime_alt', 'datetime_alt', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'date_todt', 'date', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'date_alt_todt', 'date_alt', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'datetime_todt', 'datetime', col_width_method='all')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'datetime_alt_todt', 'datetime_alt', col_width_method='all')



table_bottom_border(test_dtype_df, ecr_wb, test_dtype_sheet, header_offset=2)
table_right_border(test_dtype_df, ecr_wb, test_dtype_sheet, header_offset=2)
insert_title(test_dtype_df, ecr_wb, test_dtype_sheet, 'Data Type Test')

# save workbook
writer.save()
writer.close()

C:\Users\charl\AppData\Local\Temp\ipykernel_12032\922009572.py:6: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter('Reports/Example Clinical Report.xlsx', engine='xlsxwriter', options={'nan_inf_to_errors':True})


ValueError: None is not a valid data_format option. Valid options are: ['numeric', 'decimal_1', 'decimal_2', 'dollar', 'dollar_cents', 'percent', 'percent_1', 'percent_2', 'date', 'date_alt', 'datetime', 'datetime_alt']